In [21]:
import os
import sys
import json
import time
import datetime
import pandas as pd
import tqdm

from dotenv import load_dotenv

from intervals import Intervals

load_dotenv()


ATHLETE_ID = ""
API_KEY = ""

try:
    ATHLETE_ID = os.environ['ATHLETE_ID']
    API_KEY = os.environ['API_KEY']
except KeyError:
    print("Please set ATHLETE_ID and API_KEY in .env file")
    sys.exit(1)


print(ATHLETE_ID)
print(API_KEY)

i171109
7hqhcubmp6ikkulwi4zanoj88


In [22]:
icu = Intervals(ATHLETE_ID, API_KEY)

START_DATE = "01/01/2024"
WELLNESS_COLS = [
    "ctl_start",
    "atl_start",
    "date",
    "weight",
    "watt_kg",
]

start_date = datetime.datetime.strptime(START_DATE, "%d/%m/%Y").date()
today_date = datetime.date.today()

In [23]:
def fetch_wellness(start_date, end_date):
    data = icu.wellness(start_date, today_date)
    df = pd.DataFrame(data)
    df["date"] = pd.to_datetime(df["id"])
    df = df.drop(columns=["id"])
    df = df.sort_values(by="date")
    df['eftp'] = df['sportInfo'].apply(lambda x: x[0]['eftp'] if len(x) > 0 else None)
    df = df.drop(columns=["sportInfo"])

    df['weight'] = df['weight'].ffill().bfill()
    df['eftp'] = df['eftp'].ffill().bfill()

    df['ctl_start'] = df['ctl'].shift(1)
    df['atl_start'] = df['atl'].shift(1)

    df['watt_kg'] = df['eftp'] / df['weight']

    return df[WELLNESS_COLS]

tic = time.time()
wellness_df = fetch_wellness(start_date, today_date)
toc = time.time()
print(f"Time taken: {toc-tic:.2f}s")

Time taken: 0.41s


In [25]:

activities = icu.activities(start_date, today_date)
activities_df = pd.DataFrame(activities)
id_list = activities_df["id"].tolist()

all_streams = []

activities = icu.activities(start_date, today_date)
activities_df = pd.DataFrame(activities)
id_list = activities_df["id"].tolist()

all_streams = []

for activity_id in tqdm.tqdm(id_list):
    streams = icu.activity_streams(activity_id)
    streams = {k: v for k, v in streams.items() if v}
    all_streams.append(streams)

streams_df = pd.DataFrame(all_streams)

100%|██████████| 47/47 [00:09<00:00,  4.91it/s]


,time,watts,cadence,heartrate,distance,altitude,latlng,velocity_smooth,temp,torque
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[105, 105, 105, 106, 107, 107, 107, 107, 106, ...","[0.0, 2.2, 4.4, 5.7, 8.0, 10.2, 14.0, 16.3, 18...","[8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, 8.6, ...","[36.72382, 36.72383, 36.723843, 36.72385, 36.7...","[0.0, 0.0, 2.215, 1.903, 1.995, 2.044, 2.363, ...","[24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 2...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[105, 107, 109, 109, 106, 103, 101, 99, 99, 10...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[33.8, 33.8, 33.8, 33.8, 33.8, 33.8, 33.8, 33....","[36.72386, 36.723866, 36.723873, 36.72389, 36....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[0, 1, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17...","[0, 0, 25, 0, 0, 0, 17, 59, 93, 105, 114, 122,...","[0, 0, 28, 0, 0, 0, 27, 64, 74, 81, 83, 84, 0,...","[109, 109, 109, 109, 109, 109, 109, 109, 109, ...","[0.0, 0.0, 0.0, 2.1, 4.3, 8.7, 11.4, 13.3, 16....","[21.2, 21.2, 21.2, 21.2, 21.2, 21.2, 21.2, 21....","[36.723972, 36.72397, 36.723892, 36.723873, 36...","[0.0, 0.0, 0.0, 0.294, 0.534, 0.963, 1.138, 2....","[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1...","[0, 0, 9, 0, 0, 0, 6, 9, 12, 12, 13, 14, 0, 0,..."
3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 70, 7...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6, 1.9, 2.6, ...","[20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20.2, 20....","[36.723755, 36.723763, 36.72377, 36.723778, 36...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.008, 0.122, 0.384,...","[19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 1...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[88, 88, 87, 84, 83, 84, 83, 83, 85, 85, 89, 9...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.8, 2.2, 3.6, 4.9, ...","[42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42.8, 42....","[36.72397, 36.72397, 36.723972, 36.723976, 36....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.16, 0.436, 0.72, 0...","[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[None, None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[83, 81, 80, 79, 81, 79, 79, 78, 80, 81, 83, 8...","[0.0, 2.9, 5.8, 20.4, 25.3, 27.3, 29.3, 31.2, ...","[7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, 7.6, ...","[36.723648, 36.723637, 36.723625, 36.723713, 3...","[0.0, 0.0, 2.905, 2.905, 6.323, 5.458, 5.268, ...","[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
6,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[91, 91, 92, 94, 96, 97, 97, 97, 96, 95, 94, 9...","[0.0, 0.8, 1.7, 2.5, 3.4, 4.2, 5.9, 10.8, 16.6...","[29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29.2, 29....","[36.724037, 36.724045, 36.724052, 36.724056, 3...","[0.0, 0.82, 0.825, 0.84, 0.845, 0.848, 1.02, 1...","[21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[135, 133, 136, 147, 158, 177, 191, 182, 178, ...","[84, 82, 82, 81, 83, 84, 82, 86, 82, 82, 82, 8...","[113, 114, 114, 114, 113, 114, 114, 114, 114, ...","[0.0, 3.0, 6.7, 11.5, 15.6, 19.2, 23.0, 27.3, ...","

In [ ]:
streams_df
            
